# MSc in CSTE, CIDA option Machine learning & Big Data Assignment

### Analysis of data from an environmental sensor network using Hadoop/Spark

In [ ]:
import numpy as np
from urllib.request import urlopen
from pyspark.sql.session import SparkSession
from pyspark import SparkFiles
from pyspark.sql.functions import *
import urllib.request, json, datetime, time
from pyspark.mllib.clustering import KMeans

In [ ]:
# Allocate more memory to the driver:
MAX_MEMORY= "8g"
# Spark session builder:
spark = SparkSession.builder.config("spark.executor.memory", MAX_MEMORY).config("spark.driver.memory", MAX_MEMORY).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("OFF")
sc.uiWebUrl

In [ ]:
# Locally save instances of the data:

today = datetime.datetime.now()
url5min = 'https://data.sensor.community/static/v2/data.json'
url24h = 'https://data.sensor.community/static/v2/data.24h.json'

# sleep until specific time (e.g. 5pm) before running the next line
today = datetime.datetime.now()
exactImportTime = datetime.datetime(today.year, today.month, today.day, 17, 0, 0)
awaitingTime = exactImportTime - today
time.sleep(awaitingTime.total_seconds())

today = datetime.datetime.now()
with urllib.request.urlopen(url5min) as url:
    data5min = json.load(url)
with open('output/5min/data5min_{}-{}-{}_{}h{}.json'.format(today.year, today.month, today.day, today.hour, str(today.minute).zfill(2)), 'w') as outfile:
    json.dump(data5min, outfile)
with urllib.request.urlopen(url24h) as url:
    data24h = json.load(url)
with open('output/24h/data24h_{}-{}-{}_{}h{}.json'.format(today.year, today.month, today.day, today.hour, str(today.minute).zfill(2)), 'w') as outfile:
    json.dump(data24h, outfile)

In [ ]:
# Load data from local files and load them into Spark DataFrames:
path = 'output/24h/'
file1 = 'data24h_2022-11-3_17h00.json'
file2 = 'data24h_2022-11-4_17h00.json'
file3 = 'data24h_2022-11-5_17h00.json'
file4 = 'data24h_2022-11-6_17h00.json'
file5 = 'data24h_2022-11-7_17h00.json'
file6 = 'data24h_2022-11-8_17h00.json'

files = [file1, file2, file3, file4, file5, file6]
dfs = []

for file in files:
    spark.sparkContext.addFile(path + file)
    filename = SparkFiles.get(file)
    df = spark.read.json(filename)
    df.printSchema()
    dfs.append(df)

In [ ]:
# AQI Map:
air = {}
air[1] = ["Low", [0,16], [0,11]]
air[2] = ["Low", [17,33], [12,23]]
air[3] = ["Low", [34,50], [24,35]]
air[4] = ["Medium", [51,58], [36,41]]
air[5] = ["Medium", [59,66], [42,47]]
air[6] = ["Medium", [67,75], [48,53]]
air[7] = ["High", [76,83], [54,58]]
air[8] = ["High", [84,91], [59,64]]
air[9] = ["High", [92,100], [65,70]]
air[10] = ["Very High", [101,10000000], [71,10000000]]

# Spark implementation & tasks:

### Task 1: Identify the top 10 countries in terms of average air quality improvement over the previous 24 hours as well as the current averaged air quality indices of each. As far as possible use the country field in the sensor data to identify the country.

In [ ]:
# Preprocessing (P1 and P2 filtering):
for i in range(len(dfs)):
    print("Raw dataset count: ", dfs[i].count())
    # Explode sensordatavalues using pyspark.sql.functions.explode
    df_ = dfs[i].withColumn('sensordatavalues', explode('sensordatavalues'))
    print("Dataset count after exploding sensordatavalues: ", df_.count())
    # Remove rows that aren't P1 or P2:
    df_ = df_[df_.sensordatavalues.value_type.isin(['P1', 'P2'])]
    print("Dataset count after removing rows that aren't P1 or P2: ", df_.count())
    # Regroup sensordatavalues by record id:
    df_ = df_.groupby('id').agg(collect_list('sensordatavalues').alias('sensordatavalues'))
    # Remove the old sensordatavalues column still containing values different from P1 and P2:
    dfs[i] = dfs[i].drop('sensordatavalues')
    # Link the new sensordatavalues column to the old dataframe, on id:
    dfs[i] = dfs[i].join(df_, on='id', how='inner')
    print("Dataset count after joining the new sensordatavalues column to the old dataframe: ", dfs[i].count())

    dfs[i].show(5)
    # dfs[i].select('id','location.country','location.id', 'sensordatavalues.value_type','sensordatavalues.value').sort('location.country', 'location.id').show(5, False)

In [ ]:
for i in range(len(dfs)):
    # replace location field with country field:
    df_ = dfs[i].withColumn('location', dfs[i]['location.country'])
    # sort by country:
    df_ = df_.sort('location')
    # explode sensordatavalues:
    df_ = df_.withColumn('sensordatavalues', explode('sensordatavalues'))
    # group by country:
    df_ = df_.groupby('location').agg(collect_list('sensordatavalues').alias('sensordatavalues'))
    # Create a RDD collection of tuples (country, (P1, P2)), so that each rdd element is a combo of a country and either its P1 or P2 values. Then convert RDDs to DataFrames and join them to the original dataframe:
    df_ = df_.join(df_.rdd.map(lambda x: (x[0], [float(y['value']) for y in x[1] if y['value_type'] == 'P1'])).toDF(['location', 'P1']), on='location', how='inner')
    df_ = df_.join(df_.rdd.map(lambda x: (x[0], [float(y['value']) for y in x[1] if y['value_type'] == 'P2'])).toDF(['location', 'P2']), on='location', how='inner')
    # Create a RDD collection to calculate the average P1 and P2 values for each country, and convert RDDs to DataFrames and join them to the original dataframe:
    df_ = df_.join(df_.rdd.map(lambda x: (x[0], float(np.round(np.mean(x[2]), 2)))).toDF(['location', 'avgP1']), on='location', how='inner')
    df_ = df_.join(df_.rdd.map(lambda x: (x[0], float(np.round(np.mean(x[3]), 2)))).toDF(['location', 'avgP2']), on='location', how='inner')
    # Associate P1 and P2 avg to their respective AQI:
    df_ = df_.join(df_.rdd.map(lambda x: (x[0], [y for y in air if air[y][2][0] <= np.round(x[4]) <= air[y][2][1]][0])).toDF(['location', 'P1_AQI']), on='location', how='inner')
    df_ = df_.join(df_.rdd.map(lambda x: (x[0], [y for y in air if air[y][2][0] <= np.round(x[5]) <= air[y][2][1]][0])).toDF(['location', 'P2_AQI']), on='location', how='inner')
    # Calculate the max AQI for each country:
    df_ = df_.withColumn('maxAQI', when(df_.P1_AQI > df_.P2_AQI, df_.P1_AQI).otherwise(df_.P2_AQI))

    dfs[i] = df_
    dfs[i].show(10)

In [ ]:
# Task 1: Identify the top 10 countries in terms of average air quality improvement over the previous 24 hours as well as the current averaged air quality indices of each.
df1 = dfs[0].select('location', 'maxAQI').withColumnRenamed('maxAQI', 'maxAQI_1')
df2 = dfs[1].select('location', 'maxAQI').withColumnRenamed('maxAQI', 'maxAQI_2')
df_diff = df1.join(df2, on='location', how='inner')
df_diff = df_diff.withColumn('diffAQI', df_diff.maxAQI_2 - df_diff.maxAQI_1).select('location', 'diffAQI')
df_diff = df_diff.sort('diffAQI', ascending=True)
df_diff.show(12)

#

### Task 2: Using the geo-coordinates from the sensor data, group the data into smaller regions using an appropriate clustering algorithm. Then determine the top 50 regions in terms of air quality improvement over the previous 24 hours.

Assume task 1 is not executed. Let's re-filter the dataframes for P1 and P2 values, and then group by clusters instead of countries.

In [ ]:
for i in range(len(dfs)):
    # Same process as task 1, we filter the dataframe to keep only P1 and P2 values
    df_ = dfs[i].withColumn('sensordatavalues', explode('sensordatavalues'))
    df_ = df_[df_.sensordatavalues.value_type.isin(['P1', 'P2'])]
    dfs[i] = dfs[i].drop('sensordatavalues').join(df_.groupby('id').agg(collect_list('sensordatavalues').alias('sensordatavalues')), on='id', how='inner')
    dfs[i] = dfs[i].select('sensor.id','location.latitude','location.longitude', 'location.altitude', 'sensordatavalues').sort('sensor.id')

# TODO: Find optimal amount of clusters

# Create a RDD with the sensor id, and a tuple of the latitude and longitude, using the oldest dataframe:
rdd = dfs[0].rdd.map(lambda x: (x[0], (float(x[1]), float(x[2]))))
# Create a KMeans model with 200 clusters using latitude and longitude values from the first dataframe:
model = KMeans.train(rdd.map(lambda x: x[1]), 200, seed=23)
# (This model will be used to predict the cluster for each sensor id in the second dataframe.)

for i in range(len(dfs)):
    # Create a RDD with the sensor id, and a tuple of the latitude and longitude using the current dataframe:
    rdd = dfs[i].rdd.map(lambda x: (x[0], (float(x[1]), float(x[2]))))
    # Create a RDD collection with both the sensor id and the corresponding predicted cluster:
    rdd_clusters = rdd.map(lambda x: (x[0], model.predict(x[1])))
    # Create a RDD containing the amount of sensors in each cluster:
    rdd_sensorAmountByCluster = rdd_clusters.map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x + y)
    # Store each cluster center in a dictionary (Each cluster center is a tuple of latitude and longitude), rounded to 2 decimals:
    centers = {i: np.round(np.array(model.clusterCenters[i]), 2) for i in range(len(model.clusterCenters))}
    # Add the cluster center to the RDD collection:
    rdd_clusters = rdd_clusters.map(lambda x: (x[1], x[0], centers[x[1]].tolist()))
    # Convert RDDs to DataFrames and join them to the original dataframe:
    dfs[i] = dfs[i].join(rdd_clusters.toDF(['cluster_id', 'id', 'cluster_center']), on='id', how='inner')
    dfs[i] = dfs[i].join(rdd_sensorAmountByCluster.toDF(['cluster_id', 'sensor_amount']), on='cluster_id', how='inner')
    # Group by cluster, keeping the cluster center and the sensordatavalues:
    dfs[i] = dfs[i].groupby('cluster_id').agg(collect_list('cluster_center')[0].alias('cluster_center'), collect_list('sensor_amount')[0].alias('sensor_amount'), collect_list('sensordatavalues').alias('sensordatavalues'))
    # PROBLEM: sensordatavalues is a list of list of data values. To fix this, explode two times the sensordatavalues column:
    dfs[i] = dfs[i].withColumn('sensordatavalues', explode('sensordatavalues'))
    dfs[i] = dfs[i].withColumn('sensordatavalues', explode('sensordatavalues'))
    # Then regroup by cluster, keeping the cluster center and the sensordatavalues for each cluster id:
    dfs[i] = dfs[i].groupby('cluster_id').agg(collect_list('cluster_center')[0].alias('cluster_center'), collect_list('sensor_amount')[0].alias('sensor_amount'), collect_list('sensordatavalues').alias('sensordatavalues'))
    # Once again, just like task 1, we compute the max AQI for each cluster:
    # Create a RDD collection of tuples (country, (P1, P2)), so that each rdd element is a combo of a cluster and either its P1 or P2 values. Then convert RDDs to DataFrames and join them to the original dataframe:
    dfs[i] = dfs[i].join(dfs[i].rdd.map(lambda x: (x[0], [float(y['value']) for y in x[3] if y[2] == 'P1'])).toDF(['cluster_id', 'P1']), on='cluster_id', how='inner')
    dfs[i] = dfs[i].join(dfs[i].rdd.map(lambda x: (x[0], [float(y['value']) for y in x[3] if y[2] == 'P2'])).toDF(['cluster_id', 'P2']), on='cluster_id', how='inner')
    # Create a RDD collection to calculate the average P1 and P2 values for each cluster, and convert RDDs to DataFrames and join them to the original dataframe:
    dfs[i] = dfs[i].join(dfs[i].rdd.map(lambda x: (x[0], float(np.round(np.mean(x[4]), 2)))).toDF(['cluster_id', 'avgP1']), on='cluster_id', how='inner')
    dfs[i] = dfs[i].join(dfs[i].rdd.map(lambda x: (x[0], float(np.round(np.mean(x[5]), 2)))).toDF(['cluster_id', 'avgP1']), on='cluster_id', how='inner')
    # Associate P1 and P2 avg to their respective AQI:
    dfs[i] = dfs[i].join(dfs[i].rdd.map(lambda x: (x[0], [y for y in air if air[y][2][0] <= np.round(x[6]) <= air[y][2][1]][0])).toDF(['cluster_id', 'P1_AQI']), on='cluster_id', how='inner')
    dfs[i] = dfs[i].join(dfs[i].rdd.map(lambda x: (x[0], [y for y in air if air[y][2][0] <= np.round(x[7]) <= air[y][2][1]][0])).toDF(['cluster_id', 'P2_AQI']), on='cluster_id', how='inner')
    # Calculate the max AQI for each cluster:
    dfs[i] = dfs[i].withColumn('maxAQI', when(dfs[i].P1_AQI > dfs[i].P2_AQI, dfs[i].P1_AQI).otherwise(dfs[i].P2_AQI))
    dfs[i] = dfs[i].select('cluster_id', 'cluster_center', 'sensor_amount', 'maxAQI')
    dfs[i].show(10)

To complete task 2, let's compare two of our previously grouped-by-clusters dataframes and table the top 50 clusters whose maxAQI difference is the lowest (the lower the diffAQI is, the merrier the air quality evolution is).

In [ ]:
# Select one dataframe (here the one before the latest):
df1 = dfs[len(dfs)-2].select('cluster_id', 'cluster_center', 'sensor_amount', 'maxAQI').withColumnRenamed('maxAQI', 'maxAQI_1')
# Select a second dataframe (here the latest):
df2 = dfs[len(dfs)-1].select('cluster_id', 'maxAQI').withColumnRenamed('maxAQI', 'maxAQI_2')
# Join both dataframes on cluster_id:
df_diff = df1.join(df2, on='cluster_id', how='inner')
# Create a column named diffAQI, whose value is the relative difference between today's maxAQI, and yesterday maxAQI:
df_diff = df_diff.withColumn('diffAQI', df_diff.maxAQI_2 - df_diff.maxAQI_1).select('cluster_id', 'cluster_center', 'sensor_amount', 'diffAQI')
# Sort the dataframe by diffAQI, starting with the lowest diffAQIs:
df_diff.sort('diffAQI', ascending=True).show(50)

In [ ]:
# TODO:
# Plot each cluster's center on a world map using geopandas:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

shapefile = 'data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp'

colors = 9
cmap = 'Blues'
figsize = (16, 10)

# Create a world map:
world = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
world.columns = ['country', 'country_code', 'geometry']
# Plot the world map:
# ax = world.plot(color='white', edgecolor='black', figsize=figsize)
# Plot each cluster's center on the world map:
for i in range(len(df_diff.take(50))):
    plt.scatter(df_diff.take(50)[i][1][0], df_diff.take(50)[i][1][1], c=df_diff.take(50)[i][3], cmap=cmap, vmin=-1, vmax=1, s=1000, alpha=0.5)
# Add a colorbar:
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=-1, vmax=1))
sm._A = []
cbar = plt.colorbar(sm)
cbar.set_label('AQI difference', rotation=270, labelpad=20)
# Show the world map:
plt.show()


###

### Task 3: Calculate the longest streaks of good air quality (ie low index values) and display as a histogram.

Assume task 2 is already done. We will use the dataframes where AQI values are already calculated for each cluster.

In [ ]:
# Create a RDD with the cluster id, and a list containing 0s or 1s if the maxAQI is respectively lower or higher than 3:
rdd = dfs[0].rdd.map(lambda x: (x[0], [0 if x[3] < 4 else 1 ]))
# Create a RDD with the cluster id, the list of 0s/1s, the current streak of repetitive 0s and the max streak:
rdd_streaks = rdd.map(lambda x: (x[0], x[1], (1 if x[1][0] == 0 else 0), (1 if x[1][0] == 0 else 0)))
# Convert the RDD to a dataframe:
df = rdd_streaks.toDF(['cluster_id', 'streaks', 'current_streak', 'max_streak'])
# For all the following days (each dataframes following the first stored one)
for i in range(1, len(dfs)):
    # Create a RDD with the cluster id, and a list containing 0s or 1s if the maxAQI is respectively lower or higher than 3:
    rdd = dfs[i].rdd.map(lambda x: (x[0], [0 if x[3] < 4 else 1 ]))
    # Create a RDD with the cluster id, the list of 0s/1s and the current streak:
    rdd_streaks = rdd.map(lambda x: (x[0], x[1], (1 if x[1][len(x[1])-1] == 0 else 0)))
    # Convert the RDD containing streak information to a dataframe, and join it to the previous dataframe:
    df = df.join(rdd_streaks.toDF(['cluster_id', 'streak', 'previous_streak']), on='cluster_id', how='inner')
    # Concatenate the 0s/1s values list with the current df 0s/1s value into a single list, and drop the colomn with only one value:
    df = df.withColumn('streaks', concat('streaks', 'streak'))
    df = df.drop('streak')
    # Update the current_streak column using the previous_streak value:
    df = df.withColumn('current_streak', when(df.previous_streak == 1, df.current_streak + 1).otherwise(0))
    # Update the max_streak value using the previous_streak and the max_streak:
    df = df.withColumn('max_streak', when(df.current_streak > df.max_streak, df.current_streak).otherwise(df.max_streak))
    # Drop the current streak value:
    df = df.drop('previous_streak')
# Show the current state of streaks for each cluster id (for verification):
df.sort('cluster_id').show(10)
# Group by max_streak, show the cluster_id and the amount of clusters with that max_streak:
df = df.groupBy('max_streak').agg(count('cluster_id').alias('cluster_amount'), collect_list('cluster_id').alias('cluster_ids'))
df = df.sort('max_streak', ascending=False)
print('The clusters ids with the longest streaks of good air quality (ie low index values), as well as the amount of clusters with that streak:')
df.show(df.count())

In [ ]:
# Create histogram of the streaks:
df = df.withColumn('max_streak', df.max_streak.cast('int'))
df = df.withColumn('cluster_amount', df.cluster_amount.cast('int'))
pdf = df.sort('max_streak', ascending=True).toPandas()
pdf.plot.bar(x='max_streak', y='cluster_amount', rot=0)

In [ ]:
spark.stop()